In [5]:
%pylab
import pandas as pd
%matplotlib inline

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [ ]:
import re

line1 = '            bash-2888  [002] ....   214.858474: kmem_cache_free: call_site=c132e814 ptr=d78dd0a0\n'
line2 = '            bash-2833  [001] ....   214.858888: kmalloc: call_site=c13a0a0d ptr=d5265780 bytes_req=188 bytes_alloc=192 gfp_flags=GFP_KERNEL_ACCOUNT|__GFP_ZERO\n'

generic_regex = r"""
    \s*
    (?P<program>\S+)             # Extract program
    -
    (?P<pid>\d+)                 # Extract pid
    \s+\[
    (?P<cpu>\d+)                 # Extract cpu
    \]\s+
    (?P<irqs_disabled>.) 
    (?P<need_resched>.)
    (?P<irq_context>.)
    (?P<preempt_count>.)
    \s+
    (?P<timestamp>[\d\.]+)
    :\s+
    (?P<function>\w+)
    :\s+.+
"""

alloc_regex = r"""
    bytes_req=(?P<bytes_req>\d+)
    \s+
    bytes_alloc=(?P<bytes_alloc>\d+)
"""

common_pattern = re.compile(generic_regex, re.VERBOSE | re.MULTILINE)
alloc_pattern = re.compile(alloc_regex, re.VERBOSE | re.MULTILINE)

#print(common_pattern.match(line1).groupdict())
#print(common_pattern.match(line2).groupdict())

print(alloc_pattern.search(line2).groupdict())


In [ ]:
reader = pd.read_csv("../Samples/analytics/kmem2.csv", chunksize=10)

df = next(reader)
print(df)

df = next(reader)
print(df)

In [ ]:
import re

line1 = '            bash-2888  [002] ....   214.858474: kmem_cache_free: call_site=c132e814 ptr=d78dd0a0\n'
line2 = '            bash-2833  [001] ....   214.858888: kmalloc: call_site=c13a0a0d ptr=d5265780 bytes_req=188 bytes_alloc=192 gfp_flags=GFP_KERNEL_ACCOUNT|__GFP_ZERO\n'

generic_regex = r"""
    (?P<program>\S+)
    -
    (?P<pid>\d+)
    \s+\[
    (?P<cpu>\d+)
    \]\s+
    (?P<irqs_disabled>.)
    (?P<need_resched>.)
    (?P<irq_context>.)
    (?P<preempt_count>.)
    \s+
    (?P<timestamp>[\d\.]+)
    :\s+
    (?P<function>\w+)
    :\s+.+
"""

alloc_regex = r"""
    bytes_req=(?P<bytes_req>\d+)
    \s+
    bytes_alloc=(?P<bytes_alloc>\d+)
"""
gpattern = re.compile(generic_regex, re.VERBOSE)
alloc_pattern = re.compile(alloc_regex, re.VERBOSE)

from collections import OrderedDict


def parse_trace_log(filename):
    with open(filename) as tracefile:
        for line in tracefile:
            generic = gpattern.search(line)
            if not generic: continue
            rec = OrderedDict(generic.groupdict())
            alloc = alloc_pattern.search(line)
            if alloc:
                rec.update(alloc.groupdict())
            print(rec)


In [ ]:
# Total memory allocation

tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

#total_size = 0
#for df in tf:
#    total_size += df.bytes_alloc.sum()

total_alloc = sum(df.bytes_alloc.sum() for df in tf)
print(total_alloc)


In [13]:
# Total memory allocation

tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1000*25)

#df = tf.get_chunk()
sum(df.bytes_alloc.sum() for df in tf)
#import sys
#sys.getsizeof(df)

46668392.0

In [24]:
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1000*25)

df = tf.get_chunk()
#dict(df.groupby("cpu")["bytes_alloc"].sum().sort_values(ascending=False).head(1))
dict(df.groupby("cpu")["bytes_alloc"].sum().sort_values(ascending=False))



{0: 972768.0, 1: 1132464.0, 2: 697608.0, 3: 1016096.0}

In [28]:
a = {"aaa": 10, "bbb": 20}
b = {"aaa": 30, "bbb": 30}

#c = {"aaa": 40, "bbb": 50}
from collections import Counter
c = Counter("this is a test string")
c

Counter({' ': 4,
         'a': 1,
         'e': 1,
         'g': 1,
         'h': 1,
         'i': 3,
         'n': 1,
         'r': 1,
         's': 4,
         't': 4})

In [34]:
a = {"aaa": 10, "bbb": 20}
b = {"aaa": 30, "bbb": 30}

#c = {"aaa": 40, "bbb": 50}
from collections import Counter
c = Counter(a)
print(c)
c += Counter(b)
print(c)

Counter({'bbb': 20, 'aaa': 10})
Counter({'bbb': 50, 'aaa': 40})


In [33]:
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1000*25)

c = Counter()
for df in tf:
    c += dict(df.groupby("cpu")["bytes_alloc"].sum())
print(c.most_common()[0])

(0, 22065744.0)


In [ ]:
# Find out in which CPU maximum memory was allocated (for single dataframe)
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()

dict(df.groupby("cpu").sum()["bytes_alloc"])

In [ ]:
# Find out in which CPU maximum memory was allocated - inefficient

tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

bytes_alloc = {0: 0, 1: 0, 2: 0, 3: 0}
for df in tf:
    row = df.groupby("cpu").sum().bytes_alloc
    bytes_alloc[0] += row.get(0, 0)
    bytes_alloc[1] += row.get(1, 0)
    bytes_alloc[2] += row.get(2, 0)
    bytes_alloc[3] += row.get(3, 0)
    
print(bytes_alloc)

In [ ]:
# Find out in which CPU maximum memory was allocated - better

tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)
from collections import Counter

bytes_alloc_per_cpu = Counter()
for df in tf:
    bytes_alloc_per_cpu += Counter(dict(df.groupby("cpu").sum().bytes_alloc))
    
print(bytes_alloc_per_cpu)


In [43]:
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()

df[df["irqs_disabled"] != "."]["bytes_alloc"].count()

31

In [ ]:
cpu, alloc = bytes_alloc_per_cpu.most_common()[0]
print("Maximum allocations on CPU {} ({} bytes)".format(cpu, alloc))

In [44]:
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()

df[df["irqs_disabled"] == 'd']["bytes_alloc"].count()

31

In [45]:
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

count = 0
for df in tf:
    count += df[df.irqs_disabled == 'd']["bytes_alloc"].count()
print(count)

811


In [47]:
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()
df.max()["bytes_alloc"]

6976.0

In [ ]:
# How often were IRQs disabled using allocation

tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

#count = 0
#for df in tf:
#    count += df[df.irqs_disabled == 'd']["bytes_alloc"].count()
count = sum(df[df.irqs_disabled == 'd']["bytes_alloc"].count() \
               for df in tf)
print(count)

In [48]:
# Find out highest memory allocation
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

#df = tf.get_chunk()

#df["bytes_alloc"].max()

largest = max(df.bytes_alloc.max() for df in tf)
print(largest)

16384.0


In [ ]:
# dummy
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()

g = df.groupby("program")
g["bytes_alloc"].sum()


In [ ]:
# dummy 2
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()

#g = df.groupby("program").agg({"bytes_alloc": np.sum})
df.groupby("program")["program", "bytes_alloc"]\
           .sum()\
           .sort_values("bytes_alloc", ascending=False)\
           .head(1)


In [ ]:
# dummy 3
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()

n = pd.DataFrame({"program": [], "bytes_alloc": []})

#for df in tf:
#    r = df.groupby("program")["bytes_alloc", "program"]\
#           .sum()\
#           .sort_values("bytes_alloc", ascending=False)\
#           .head(1)
#    c = dict(r)
#    print(c)

r = df.groupby("program")["bytes_alloc", "program"]\
           .sum()\
           .sort_values("bytes_alloc", ascending=False)\
           .head(1).reset_index()

#print(type(r))
#n.append(r, ignore_index=True)
#n
pd.concat([n, r])

In [ ]:
# dummy 4
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()

#for df in tf:
#    r = df.groupby("program")["bytes_alloc", "program"]\
#           .sum()\
#           .sort_values("bytes_alloc", ascending=False)\
#           .head(1)
#    c = dict(r)
#    print(c)

r = df.groupby("program")["bytes_alloc", "program"]\
           .sum()\
           .sort_values("bytes_alloc", ascending=False)\
           .head(1).reset_index()
r


In [ ]:
# Find out which program was responsible for maximum allocations
# Complicated!
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

n = pd.DataFrame({"program": [], "bytes_alloc": []})
#c = Counter()
for df in tf:
    r = df.groupby("program")["bytes_alloc", "program"]\
             .sum()\
             .sort_values("bytes_alloc", ascending=False)\
             .head(1)\
             .reset_index()
    n = pd.concat([n, r])

f = n.groupby("program")["bytes_alloc", "program"]\
    .sum()\
    .sort_values("bytes_alloc", ascending=False)\
    .head(1)\
    .reset_index()
f

In [51]:
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = next(tf)
df[df.function == "kfree"].count()

program          4699
pid              4699
cpu              4699
irqs_disabled    4699
need_resched     4699
irq_context      4699
preempt_count    4699
timestamp        4699
function         4699
bytes_req           0
bytes_alloc         0
dtype: int64

In [ ]:
# Find out which program was responsible for maximum allocations
# Easier
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)
from collections import Counter

bytes_alloc_per_program = Counter()
for df in tf:
    bytes_alloc_per_program += Counter(dict(df.groupby("program").sum().bytes_alloc))
    
print(bytes_alloc_per_program.most_common()[0])



In [55]:
results = (True, True, True, True)
all(results)

True

In [96]:
# Find out if the number of kmallocs() is lesser than or equal to kfrees()

tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

#df = tf.get_chunk()

#all([df[df.function == "kmalloc"]["function"].count() \
#       <= df[df.function == "kfree"]["function"].count() for df in tf])

c = Counter()

for df in tf:
    c += Counter(df["function"])

#c = Counter(df["function"])
#c

c["kfree"] > c["kmalloc"]

True

In [97]:
# Find out if the number of kmallocs() is lesser than or equal to kfrees()

tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()

df

,program,pid,cpu,irqs_disabled,need_resched,irq_context,preempt_count,timestamp,function,bytes_req,bytes_alloc
0,bash,2888,2,.,.,.,.,214.858474,kmem_cache_free,NaN,NaN
1,bash,2888,2,.,.,.,.,214.858475,kmem_cache_free,NaN,NaN
2,bash,2888,2,.,.,.,.,214.858476,kmem_cache_free,NaN,NaN
3,bash,2888,2,.,.,.,.,214.858476,kmem_cache_free,NaN,NaN
4,bash,2888,2,.,.,.,.,214.858477,kmem_cache_free,NaN,NaN
5,bash,2888,2,.,.,.,.,214.858478,kmem_cache_free,NaN,NaN
6,bash,2888,2,.,.,.,.,214.858478,kmem_cache_free,NaN,NaN
7,bash,2888,2,.,.,.,.,214.858480,kmem_cache_free,NaN,NaN
8,bash,2888,2,.,.,.,.,214.858481,kmem_cache_free,NaN,NaN
9,bash,2888,2,.,.,.,.,214.858481,kmem_cache_free,NaN,NaN


In [102]:
# Find out the total size of internal fragmentation
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()

df["fragmentation"] = df["bytes_alloc"] - df["bytes_req"]
print(df["fragmentation"].sum())


40615.0


In [ ]:
# Find out the total size of internal fragmentation
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)


fragmentation = 0
for df in tf:
    df["fragmentation"] = df["bytes_alloc"] - df["bytes_req"]
    fragmentation += df["fragmentation"].sum()
    
print(fragmentation)

In [ ]:
# dummy
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)


#print(df.columns)
df = tf.get_chunk()
df.replace({".": False, 
            "d": True, 
            "n": True, 
            "N": True, 
            "s": True, 
            "h": True}, inplace=True)


In [103]:
# Find out which process is responsible for most preemption disabled states
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)


#print(df.columns)
df = tf.get_chunk()
df[df.preempt_count != "."].groupby("program").count().sort_values("pid", ascending=False)

,pid,cpu,irqs_disabled,need_resched,irq_context,preempt_count,timestamp,function,bytes_req,bytes_alloc
program,,,,,,,,,,
<idle>,670,670,670,670,670,670,670,670,0,0
bash,281,281,281,281,281,281,281,281,18,18
git,190,190,190,190,190,190,190,190,10,10
ls,98,98,98,98,98,98,98,98,2,2
cat,18,18,18,18,18,18,18,18,1,1
screen,14,14,14,14,14,14,14,14,0,0
automount,10,10,10,10,10,10,10,10,0,0


In [ ]:
# Find out total memory allocation on per-process basis
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()
df.groupby("program").sum()["bytes_alloc"].sort_values(ascending=False)

In [106]:
from statistics import median
median()

['Decimal',
 'Fraction',
 'StatisticsError',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_coerce',
 '_convert',
 '_counts',
 '_exact_ratio',
 '_fail_neg',
 '_find_lteq',
 '_find_rteq',
 '_isfinite',
 '_ss',
 '_sum',
 'bisect_left',
 'bisect_right',
 'chain',
 'collections',
 'decimal',
 'groupby',
 'harmonic_mean',
 'math',
 'mean',
 'median',
 'median_grouped',
 'median_high',
 'median_low',
 'mode',
 'numbers',
 'pstdev',
 'pvariance',
 'stdev',
 'variance']

In [109]:
# Find out total memory allocation on per-process basis
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)
c = Counter()

for df in tf:
    c += Counter(dict(df.groupby("program").median()["bytes_alloc"]))
 

df = next(tf)
g = df.groupby("program")
g.median()

,pid,cpu,timestamp,bytes_req,bytes_alloc
program,,,,,
<idle>,0,1,222.976181,NaN,NaN
automount,2457,0,220.383056,NaN,NaN
bash,2900,2,223.603884,96.0,96.0
cat,2893,3,218.524920,96.0,96.0
dbus-daemon,2206,0,221.699498,320.0,320.0
git,2904,1,222.937675,104.0,112.0
kworker/u8:5,1156,3,219.647035,98.0,112.0
kworker/u8:7,1744,2,223.805003,192.0,192.0
ls,2905,3,223.601663,506.0,608.0


In [110]:
# Find out total memory allocation on per-process basis
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)


from statistics import median

d = {}

for df in tf:
    d.update
    print(dict(df.groupby("program").median()["bytes_alloc"]))
 

#df = next(tf)
#g = df.groupby("program")
#g.median()

{'<idle>': nan, 'automount': nan, 'bash': 96.0, 'cat': 96.0, 'dbus-daemon': 320.0, 'git': 112.0, 'kworker/u8:5': 112.0, 'kworker/u8:7': 192.0, 'ls': 608.0, 'memballoon': nan, 'screen': nan, 'timesync': nan, 'vminfo': 160.0, 'vmstats': nan}
{'<idle>': 144.0, 'automount': nan, 'bash': 96.0, 'cat': 32.0, 'dbus-daemon': 320.0, 'df': 576.0, 'git': 96.0, 'jbd2/sda1-8': 64.0, 'kworker/u8:5': 128.0, 'kworker/u8:7': 112.0, 'memballoon': nan, 'screen': nan, 'timesync': nan, 'vminfo': 160.0, 'vmstats': nan}
{'automount': nan, 'cat': 32.0}
{'cat': 32.0, 'vmstats': nan}
{'automount': nan, 'cat': 32.0}
{'cat': 32.0, 'screen': nan}
{'<...>': 256.0, '<idle>': nan, 'automount': nan, 'bash': 96.0, 'cat': 32.0, 'kworker/u8:7': 112.0, 'screen': 96.0}
{'<idle>': nan, 'bash': 96.0, 'cat': 32.0, 'dirname': 96.0, 'git': 96.0, 'modprobe': 96.0, 'screen': nan, 'tty': 96.0, 'uname': 96.0}
{'automount': nan, 'cat': 32.0}
{'cat': 32.0}
{'automount': nan, 'bash': nan, 'cat': 32.0, 'memballoon': nan, 'screen': 1024.

In [115]:
# Find out total memory allocation on per-process basis
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1000)

print(sys.getsizeof(tf))
df = next(tf)
print(sys.getsizeof(df))

56
404617


In [113]:
# Find out total memory allocation on per-process basis
df = pd.read_csv("../Samples/analytics/kmem-trace.csv")

df.to_excel()
df.count()
import sys
sys.getsizeof(df)

613605233

In [4]:
# Find out what size was most requested for kmalloc()
tf = pd.read_csv("../Samples/analytics/kmem-trace.csv", chunksize=1024*25)

df = tf.get_chunk()
df.groupby("program").sum()["bytes_alloc"].sort_values(ascending=False)

program
bash            1748312.0
git             1389880.0
ls               617872.0
dbus-daemon      103488.0
cat               40992.0
vminfo            35456.0
kworker/u8:5       1120.0
kworker/u8:7        192.0
vmstats               0.0
timesync              0.0
screen                0.0
memballoon            0.0
automount             0.0
<idle>                0.0
Name: bytes_alloc, dtype: float64

In [141]:
data1 = [
 {'name': 'john',   'maths': 78, 'compsci': 89 },
 {'name': 'adrian', 'maths': 73, 'compsci': 91 },
 {'name': 'bourne', 'maths': 94, 'compsci': 86 },
 {'name': 'smith',  'maths': 65, 'compsci': 68 }]
 
data2 = [
 {'name': 'john',   'physics': 79  },
 {'name': 'adrian',  'physics': 84  },
 {'name': 'bourne',  'physics': 80  },
 {'name': 'smith',  'physics': 87  },
 {'name': 'gary',   'physics': 78  }]

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

df3 = pd.concat([df1, df2], axis=1)
df3

,compsci,maths,name,name,physics
0,89.0,78.0,john,john,79
1,91.0,73.0,adrian,adrian,84
2,86.0,94.0,bourne,bourne,80
3,68.0,65.0,smith,smith,87
4,NaN,NaN,NaN,gary,78


In [150]:
data1 = [
 {'name': 'john',   'maths': 78, 'compsci': 89 },
 {'name': 'adrian', 'maths': 73, 'compsci': 91 },
 {'name': 'bourne', 'maths': 94, 'compsci': 86 },
 {'name': 'smith',  'maths': 65, 'compsci': 68 }]
 
data2 = [
 {'name': 'john',   'physics': 79  },
 {'name': 'adrian',  'physics': 84  },
 {'name': 'bourne',  'physics': 80  },
 {'name': 'smith',  'physics': 87  },
 {'name': 'gary',   'physics': 78  }]

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

#df1.join(df2, how="outer")
#df1
df3 = df1.merge(df2)
df3
#df1
#pd.merge(df1, df2)
df1.merge()

,compsci,maths,name,physics
0,89,78,john,79
1,91,73,adrian,84
2,86,94,bourne,80
3,68,65,smith,87


In [ ]:
with open("../Samples/analytics/kmem-trace.csv") as infile:
    for line in infile:
        rec = line.split(",")
        rec[0]